# Explore here

In [44]:
# Your code here

import pandas as pd
import seaborn as sns
import nltk
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split




In [7]:
df = pd.read_csv("hf://datasets/PrkhrAwsti/Twitter_Sentiment_3M/twitter_dataset.csv")

In [8]:
download("wordnet")
lemmatizer = WordNetLemmatizer()
download("stopwords")
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\luige\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luige\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [9]:

def limpiar_tweet(text):
    # Convertir a minúsculas
    text = text.lower()
    
    # Eliminar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    
    # Eliminar menciones (@usuario)
    text = re.sub(r'@\w+', '', text)
    
    # Eliminar caracteres que no sean letras, números, espacios, o hashtags
    text = re.sub(r'[^a-z0-9# ]', ' ', text)
    
    # Eliminar letras sueltas (que suelen ser ruido)
    text = re.sub(r'\s+[a-z]\s+', ' ', text)
    
    # Reducir espacios múltiples a uno
    text = re.sub(r'\s+', ' ', text).strip()

    return text.split()


In [10]:
def lemmatizar_text(tweets, lemmatizer=lemmatizer, stop_words=stop_words):
    
    # Lematizar los tweets
    tokens = [lemmatizer.lemmatize(tweet) for tweet in tweets]
    
    # Eliminar stopwords
    tokens = [tweet for tweet in tokens if tweet not in stop_words]
    
    # Eliminar tweets con longitud menor o igual a 3 caracteres
    tokens = [tweet for tweet in tokens if len(tweet) > 3]
    
    return tokens

In [11]:
df

,Unnamed: 0,tweet,sentiment
0,0,is upset that he can't update his Facebook by ...,0.0
1,1,@Kenichan I dived many times for the ball. Man...,0.0
2,2,my whole body feels itchy and like its on fire,0.0
3,3,"@nationwideclass no, it's not behaving at all....",0.0
4,4,@Kwesidei not the whole crew,0.0
...,...,...,...
3142398,3142398,"With ad revenues falling, what’s the impact on...",2.0
3142399,3142399,RT @KelvinSCWong: Well another point to add to...,2.0
3142400,3142400,$ITOX working on a contract with a fortune 500...,2.0
3142401,3142401,"$DIS it could break the 120 pin, then 125&gt; ...",2.0


In [12]:
df.head()

,Unnamed: 0,tweet,sentiment
0,0,is upset that he can't update his Facebook by ...,0.0
1,1,@Kenichan I dived many times for the ball. Man...,0.0
2,2,my whole body feels itchy and like its on fire,0.0
3,3,"@nationwideclass no, it's not behaving at all....",0.0
4,4,@Kwesidei not the whole crew,0.0


In [13]:
df.tail()

,Unnamed: 0,tweet,sentiment
3142398,3142398,"With ad revenues falling, what’s the impact on...",2.0
3142399,3142399,RT @KelvinSCWong: Well another point to add to...,2.0
3142400,3142400,$ITOX working on a contract with a fortune 500...,2.0
3142401,3142401,"$DIS it could break the 120 pin, then 125&gt; ...",2.0
3142402,3142402,Amedisys Inc $AMED COO Christopher Gerard Sell...,2.0


In [14]:
df=df.drop(columns=["Unnamed: 0"],axis=1)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142403 entries, 0 to 3142402
Data columns (total 2 columns):
 #   Column     Dtype  
---  ------     -----  
 0   tweet      object 
 1   sentiment  float64
dtypes: float64(1), object(1)
memory usage: 47.9+ MB


In [16]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
sentiment,3142321.0,0.503761,0.506766,0.0,0.0,1.0,1.0,2.0


In [17]:
ds=df[df['sentiment']>1]
ds

,tweet,sentiment
3123980,Top 3 all @Apple #tablets. Damn right! http://...,2.0
3123981,CNBCTV: #Apple's margins better than expected?...,2.0
3123987,RT @peterpham: Bought my @AugustSmartLock at t...,2.0
3124002,Latest Apple Products Leading in Efficiency ht...,2.0
3124007,RT @saxonidubom: @rwang0 @Apple Thanks...think...,2.0
...,...,...
3142398,"With ad revenues falling, what’s the impact on...",2.0
3142399,RT @KelvinSCWong: Well another point to add to...,2.0
3142400,$ITOX working on a contract with a fortune 500...,2.0
3142401,"$DIS it could break the 120 pin, then 125&gt; ...",2.0


In [18]:
df.isnull().sum()

tweet         0
sentiment    82
dtype: int64

In [19]:
df=df.dropna()
df

,tweet,sentiment
0,is upset that he can't update his Facebook by ...,0.0
1,@Kenichan I dived many times for the ball. Man...,0.0
2,my whole body feels itchy and like its on fire,0.0
3,"@nationwideclass no, it's not behaving at all....",0.0
4,@Kwesidei not the whole crew,0.0
...,...,...
3142398,"With ad revenues falling, what’s the impact on...",2.0
3142399,RT @KelvinSCWong: Well another point to add to...,2.0
3142400,$ITOX working on a contract with a fortune 500...,2.0
3142401,"$DIS it could break the 120 pin, then 125&gt; ...",2.0


In [20]:
df['sentiment'] = df['sentiment'].astype(int)

In [21]:
df[df['tweet']==""]

,tweet,sentiment


In [22]:
df=df[df["tweet"]!=""]
df      

,tweet,sentiment
0,is upset that he can't update his Facebook by ...,0
1,@Kenichan I dived many times for the ball. Man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@Kwesidei not the whole crew,0
...,...,...
3142398,"With ad revenues falling, what’s the impact on...",2
3142399,RT @KelvinSCWong: Well another point to add to...,2
3142400,$ITOX working on a contract with a fortune 500...,2
3142401,"$DIS it could break the 120 pin, then 125&gt; ...",2


In [23]:
df=df[df["sentiment"]!=2]
df

,tweet,sentiment
0,is upset that he can't update his Facebook by ...,0
1,@Kenichan I dived many times for the ball. Man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@Kwesidei not the whole crew,0
...,...,...
3138695,Q3 2020 EPS Estimates for Ball Co. $BLL Increa...,1
3138698,#stocks back from the recovery room: https://t...,1
3138699,RT @MacroCharts: Breadth – expanding last week...,1
3138701,$AAPL $QQQ Top may now be in. https://t.co/iNK...,1


In [24]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
sentiment,3131596.0,0.498637,0.499998,0.0,0.0,0.0,1.0,1.0


In [25]:
df = df.copy()
df['proTweets']=df['tweet'].apply(limpiar_tweet)
df

,tweet,sentiment,proTweets
0,is upset that he can't update his Facebook by ...,0,"[is, upset, that, he, can, update, his, facebo..."
1,@Kenichan I dived many times for the ball. Man...,0,"[dived, many, times, for, the, ball, managed, ..."
2,my whole body feels itchy and like its on fire,0,"[my, whole, body, feels, itchy, and, like, its..."
3,"@nationwideclass no, it's not behaving at all....",0,"[no, it, not, behaving, at, all, m, mad, why, ..."
4,@Kwesidei not the whole crew,0,"[not, the, whole, crew]"
...,...,...,...
3138695,Q3 2020 EPS Estimates for Ball Co. $BLL Increa...,1,"[q3, 2020, eps, estimates, for, ball, co, bll,..."
3138698,#stocks back from the recovery room: https://t...,1,"[#stocks, back, from, the, recovery, room, fai..."
3138699,RT @MacroCharts: Breadth – expanding last week...,1,"[rt, breadth, expanding, last, week, discussio..."
3138701,$AAPL $QQQ Top may now be in. https://t.co/iNK...,1,"[aapl, qqq, top, may, now, be, in]"


In [26]:
df=df.drop(columns=["tweet"],axis=1)

In [27]:
df['lemantizedTweets']=df['proTweets'].apply(lemmatizar_text)
df

,sentiment,proTweets,lemantizedTweets
0,0,"[is, upset, that, he, can, update, his, facebo...","[upset, update, facebook, texting, might, resu..."
1,0,"[dived, many, times, for, the, ball, managed, ...","[dived, many, time, ball, managed, save, rest,..."
2,0,"[my, whole, body, feels, itchy, and, like, its...","[whole, body, feel, itchy, like, fire]"
3,0,"[no, it, not, behaving, at, all, m, mad, why, ...",[behaving]
4,0,"[not, the, whole, crew]","[whole, crew]"
...,...,...,...
3138695,1,"[q3, 2020, eps, estimates, for, ball, co, bll,...","[2020, estimate, ball, increased, analyst, #st..."
3138698,1,"[#stocks, back, from, the, recovery, room, fai...","[#stocks, back, recovery, room, fair, value, f..."
3138699,1,"[rt, breadth, expanding, last, week, discussio...","[breadth, expanding, last, week, discussion, h..."
3138701,1,"[aapl, qqq, top, may, now, be, in]",[aapl]


In [28]:
df=df.drop(columns=["proTweets"],axis=1)

In [43]:
lista_tweets=df['lemantizedTweets']
lista_tweets = [" ".join(tweet) for tweet in lista_tweets]

vectorizer = TfidfVectorizer(max_features = 5000, max_df = 0.8, min_df = 5)
X = vectorizer.fit_transform(lista_tweets)
y = df["sentiment"]

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 11862616 stored elements and shape (2505276, 5000)>

In [49]:
X_train.shape

(2505276, 5000)